# Nearmap AI Feature Extraction — Parcel-Level Enrichment

This notebook processes the USC Nearmap sample imagery and AI-detection layers to produce a **parcel-level feature table** for the Hidden-Housing study area.

**Pipeline overview**

| Step | Description |
|------|-------------|
| 1 | Load Nearmap building footprints and AI feature layers |
| 2 | Load LA City administrative data (landbase, assessor, LARIAC) |
| 3 | Reproject all layers to a common CRS (EPSG:2229) |
| 4 | Estimate and correct a systematic spatial offset in the Nearmap data |
| 5 | Compute per-parcel area coverage for buildings and each AI feature |
| 6 | Attach assessor attributes (property type, year built, units, etc.) |
| 7 | Save the enriched parcel table to GeoPackage |


## Step 1 — Load Nearmap Data

Load the Nearmap tile boundary (used as the study-area mask) and building-footprint layer. Tile boundaries are in EPSG:3857; everything will be reprojected later.

In [ ]:
import geopandas as gpd
import glob, re, os

import matplotlib.pyplot as plt
import numpy as np
from shapely.affinity import translate
from sklearn.neighbors import BallTree

from src.geoadmin import load_laraic, get_landbase_bymask, load_assessor_parcels_bygeom

os.chdir('../..')

# ── Paths ──────────────────────────────────────────────────────────────────
NEARMAP_DIR  = '/Users/adamswietek/Downloads/USC_Nearmap_Sample_Data'
NEARMAP_TILE = f'{NEARMAP_DIR}/IMAGERY_SAMPLE_EPSG3857_Date20251005/Tiles.shp'
NEARMAP_AI   = f'{NEARMAP_DIR}/AI_SAMPLE'
NEARMAP_BLDG = f'{NEARMAP_AI}/ai_features_None_Building.gpkg'

nearmap_building = gpd.read_file(NEARMAP_BLDG)
nearmap_tile     = gpd.read_file(NEARMAP_TILE)

### Discover AI Feature Layers

Scan the AI sample directory for all per-feature GeoPackage files. The `_Building` layer is handled separately (loaded above); the remaining 20 layers (roof type, condition flags, tree overhang, etc.) are processed in bulk in Step 5.

In [ ]:
layer_files = sorted(glob.glob(f'{NEARMAP_AI}/ai_features_None_*.gpkg'))
skip = re.compile(r'_Building.*|_Deprecated.*', re.IGNORECASE)
layer_files = [f for f in layer_files if not skip.search(os.path.basename(f))]

def layer_name(path):
    """ai_features_None_Roof_Rusting.gpkg  →  roof_rusting"""
    name = os.path.basename(path).replace('ai_features_None_', '').replace('.gpkg', '')
    return name.lower().replace('-', '_')

ai_cols = [layer_name(f) for f in layer_files]
print(f"Found {len(layer_files)} AI feature layers:")
for col in ai_cols:
    print(f"  {col}")

## Step 2 — Load LA City Administrative Data

Three datasets are loaded for the study area, clipped to the Nearmap tile footprint:

- **Landbase** — parcel polygons with a stable `ASSETID` key
- **Assessor** (APD) — roll-year attributes: property type, year built, units, value, etc.
- **LARIAC** — building footprints derived from aerial lidar (used for height / area reference)

All three are loaded for the 2020 roll year, which best matches the Nearmap imagery date (Oct 2025 imagery; 2020 is the most recent available assessor snapshot).

In [ ]:
LANDBASE_YR = 2020
ASSESSOR_YR = 2020
LARIAC_YR   = 2020

def load_data_by_year(neighborhood, lb_yr, apd_yr, lar_yr, source='lariac'):
    landbase_parcels = get_landbase_bymask(neighborhood, lb_yr)

    polygon_geometry = neighborhood.to_crs(4326).iloc[0].geometry
    assessor_data = load_assessor_parcels_bygeom(polygon_geometry)
    assessor_data = assessor_data.loc[assessor_data.RollYear == apd_yr]
    assessor_data = assessor_data.to_crs(landbase_parcels.crs)

    if source == 'lariac':
        lariac_structures = load_laraic(neighborhood, lar_yr)
        lariac_structures = lariac_structures.to_crs(landbase_parcels.crs)

    return landbase_parcels, assessor_data, lariac_structures

aoi = nearmap_tile.dissolve()
landbase, assessor, lariac = load_data_by_year(
    aoi, lb_yr=LANDBASE_YR, apd_yr=ASSESSOR_YR, lar_yr=LARIAC_YR)

## Step 3 — Prepare Data Slices and Reproject

Slim each dataset down to the columns needed downstream, then reproject the Nearmap building footprints from EPSG:3857 to the landbase CRS (EPSG:2229 — NAD83 California State Plane Zone V, US survey feet). All subsequent spatial operations use this projected CRS.

In [ ]:
lb  = landbase[['ASSETID', 'geometry']].copy()
asr = assessor.copy()
lar = lariac.dropna(subset=['HEIGHT'])[['BLD_ID', 'AREA', 'geometry']].copy()

# Reproject Nearmap buildings to the landbase CRS (EPSG:2229, NAD83 California zone 5 ft-US)
nm_proj = (nearmap_building[['id', 'areaSqft', 'clippedAreaSqft', 'geometry']]
           .copy()
           .to_crs(lb.crs))
print(f"nm_proj: {len(nm_proj):,} buildings  |  CRS: {nm_proj.crs}")

## Step 4 — Estimate and Correct Spatial Offset

Nearmap AI features are often shifted slightly relative to authoritative parcel boundaries due to differences in georeferencing. We estimate the shift by:

1. Computing centroids for all Nearmap buildings and all landbase parcels.
2. Finding the nearest-neighbour landbase parcel for each Nearmap building (pairs farther than 200 ft are discarded).
3. Taking the **median** centroid displacement as the correction offset.

The same `(OFFSET_X, OFFSET_Y)` is then applied to the building footprints **and** every AI feature layer.

In [ ]:
# Estimate the systematic spatial shift between Nearmap buildings and landbase parcel centroids
# using nearest-neighbour matching. Only pairs within 200 ft are used to exclude bad matches.
lb_xy = np.column_stack([lb.geometry.centroid.x, lb.geometry.centroid.y])
nm_xy = np.column_stack([nm_proj.geometry.centroid.x, nm_proj.geometry.centroid.y])

tree = BallTree(lb_xy)
dist, idx = tree.query(nm_xy, k=1)

mask = dist[:, 0] < 200   # within 200 ft
dx = lb_xy[idx[mask, 0], 0] - nm_xy[mask, 0]
dy = lb_xy[idx[mask, 0], 1] - nm_xy[mask, 1]

OFFSET_X = float(np.median(dx))
OFFSET_Y = float(np.median(dy))

print(f"Pairs used : {mask.sum()}")
print(f"  dx  mean={dx.mean():.2f}  median={OFFSET_X:.2f}  std={dx.std():.2f}  ft")
print(f"  dy  mean={dy.mean():.2f}  median={OFFSET_Y:.2f}  std={dy.std():.2f}  ft")
print(f"\nApplied offset  →  dx={OFFSET_X:.2f} ft,  dy={OFFSET_Y:.2f} ft")

In [ ]:
# Visualise a 500 ft buffer around the first building to inspect the correction
sample_pt = nm_proj.geometry.iloc[0].centroid
buf = 500
xmin, ymin, xmax, ymax = sample_pt.x-buf, sample_pt.y-buf, sample_pt.x+buf, sample_pt.y+buf

nm_aligned_viz = nm_proj.copy()
nm_aligned_viz.geometry = nm_aligned_viz.geometry.translate(xoff=OFFSET_X, yoff=OFFSET_Y)

fig, axes = plt.subplots(1, 2, figsize=(16, 8))
for ax, nm_plot, title in zip(
    axes,
    [nm_proj.cx[xmin:xmax, ymin:ymax],
     nm_aligned_viz.cx[xmin:xmax, ymin:ymax]],
    ['Before correction', f'After correction  (dx={OFFSET_X:.1f} ft, dy={OFFSET_Y:.1f} ft)']
):
    lb.cx[xmin:xmax, ymin:ymax].plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.5)
    nm_plot.plot(ax=ax, facecolor='steelblue', alpha=0.4, edgecolor='steelblue', linewidth=1)
    ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax)
    ax.set_title(title, fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Apply the offset to all Nearmap buildings — this is the authoritative aligned layer
nm = nm_proj.copy()
nm.geometry = nm.geometry.translate(xoff=OFFSET_X, yoff=OFFSET_Y)
nm = nm[['id', 'areaSqft', 'clippedAreaSqft', 'geometry']]
print(f"nm (aligned): {len(nm):,} buildings  |  CRS: {nm.crs}")

## Step 5 — Extract Parcel-Level Features

For each aligned layer (buildings + 20 AI features):

1. **Dissolve** all polygons into a single geometry (avoids double-counting overlapping detections).
2. **Overlay** (intersection) with the landbase parcel grid.
3. Sum the intersection area per `ASSETID` to get the **square footage** of that feature within each parcel.

The result is one row per parcel with columns `nm_bldg_sqft`, `roof`, `gable`, `shingle`, `structural_damage`, etc.

In [ ]:
# Dissolve aligned building footprints into a single geometry, then intersect with each parcel.
# The intersection area gives the square footage of Nearmap building coverage per parcel.
nm_diss = nm[['geometry']].dissolve()
nm_over = gpd.overlay(nm_diss, lb[['ASSETID', 'geometry']], how='intersection')
nm_over['nm_bldg_sqft'] = nm_over.geometry.area

parcel_features = nm_over[['ASSETID', 'nm_bldg_sqft', 'geometry']].copy()
print(f"Parcels with Nearmap building coverage: {len(parcel_features):,}")

# For each AI feature layer: apply the same spatial offset, dissolve, overlay, and compute area.
print("\nProcessing AI feature layers...")
for fpath in layer_files:
    col = layer_name(fpath)
    layer = gpd.read_file(fpath)[['geometry']].to_crs(lb.crs)
    layer.geometry = layer.geometry.translate(xoff=OFFSET_X, yoff=OFFSET_Y)
    layer_diss = layer.dissolve()
    layer_over = gpd.overlay(layer_diss, lb[['ASSETID', 'geometry']], how='intersection')
    layer_over[col] = layer_over.geometry.area
    agg = layer_over.groupby('ASSETID')[col].sum().reset_index()
    parcel_features = parcel_features.merge(agg, on='ASSETID', how='left')
    n = parcel_features[col].notna().sum()
    print(f"  {col:<45}  {n:>4} parcels")

## Step 6 — Attach Assessor Attributes

Spatially join the assessor parcel data to the landbase parcel grid, then merge those attributes into the Nearmap feature table on `ASSETID`. This adds property-level information (AIN, property type, year built, square footage, number of units, assessed value, etc.).

In [ ]:
# Spatial join of landbase parcels with assessor parcel data.
# A small buffer(0) is applied to landbase geometries to fix any invalid geometry issues.
lb['geometry'] = lb.buffer(0)
parcel_asr = (gpd.sjoin(lb, asr, how='left', predicate='intersects')
                 .drop(columns=['index_right', 'geometry'], errors='ignore'))

In [ ]:
parcel_features = parcel_features.merge(parcel_asr, on='ASSETID', how='left')

# Normalise AIN column name (case-insensitive lookup)
ain_col = next((c for c in parcel_features.columns if c.upper() == 'AIN'), None)
if ain_col and ain_col != 'AIN':
    parcel_features = parcel_features.rename(columns={ain_col: 'AIN'})

print(f"parcel_features: {len(parcel_features):,} rows")
print(f"  with AIN         : {parcel_features['AIN'].notna().sum():,}")
print(f"  with nm_bldg_sqft: {parcel_features['nm_bldg_sqft'].notna().sum():,}")
parcel_features[['ASSETID', 'AIN', 'nm_bldg_sqft'] + ai_cols].head(3)

## Step 7 — Save Outputs

Write the enriched parcel table and the assessor data to GeoPackage files in `data/processed/` for downstream modelling and mapping.

In [ ]:
asr.to_file('data/processed/asr.gpkg')
parcel_features.to_file('data/processed/nm_parcel_joined.gpkg')
print("Saved:"  )
print("  data/processed/asr.gpkg")
print("  data/processed/nm_parcel_joined.gpkg")

## Step 8 — Preview Final Dataset

Quick sanity check on the final parcel table.

In [ ]:
# Quick look at key attributes in the final parcel table
parcel_features[['ASSETID', 'AIN', 'RollYear', 'PropertyType', 'YearBuilt',
                 'SQFTmain', 'Units', 'nm_bldg_sqft'] + ai_cols].head(5)